<a href="https://colab.research.google.com/github/AbolajiOsobamiro/pythoncodes/blob/main/Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

from keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images/255.0, test_images/255.0

class_names = ['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
IMG_INDEX=49999
plt.imshow(train_images[IMG_INDEX], cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[IMG_INDEX][0]])
plt.show()

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

In [ ]:
model.summary()

In [6]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'])


history = model.fit(train_images, train_labels,epochs=10,
                    validation_data=(test_images,test_labels))


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(test_acc)

In [10]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [14]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [19]:
test_img=train_images[300]
img=image.img_to_array(test_img)
img=img.reshape((1,)+img.shape)

In [ ]:
i=0

for batch in datagen.flow(img,save_prefix='test',save_format='jpeg'):
  plt.figure(i)
  plot = plt.imshow(image.img_to_array(batch[0]))
  i+=1
  if i > 4:
    break

plt.show()

In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

In [ ]:
get_label_name=metadata.features['label'].int2str

for image, label in raw_train.take(4):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [34]:
IMG_SIZE=160

def format_example(image,label):
  image=tf.cast(image, tf.float32)
  image=(image/127.5) - 1
  image=tf.image.resize(image, (IMG_SIZE,IMG_SIZE))
  return image,label

In [43]:
train=raw_train.map(format_example)
validation=raw_validation.map(format_example)
test=raw_test.map(format_example)

In [ ]:
for image, label in train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [52]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)

base_model= tf.keras.applications.MobileNetV2(
    input_shape= IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [ ]:
base_model.summary()

In [57]:
batch_size = 32
shuffle_buffer_size= 1000

train_batches = train.shuffle(shuffle_buffer_size).batch(batch_size)
validation_batches = validation.batch(batch_size)
test_batches = test.batch(batch_size)

In [61]:
for image, _ in train_batches.take(1):
  pass

feature_batch=base_model(image)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [66]:
base_model.trainable=False

In [63]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [64]:
prediction_layer = keras.layers.Dense(1)

In [74]:
model=tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer])

In [ ]:
model.summary()

In [77]:
base_learning_rate=0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [78]:
initial_epochs=3
validation_steps=20

loss0,accuracy0=(model.evaluate(validation_batches, steps=validation_steps))

20/20 [==============================] - 12s 505ms/step - loss: 0.7507 - accuracy: 0.5172


In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

accuracy=history.history['accuracy']
print(accuracy)

In [84]:
model.save('dogs_vs_cats.h5')
new_model=tf.keras.models.load_model('dogs_vs_cats.h5')